# Búsqueda en Profundidad Iterativa (Iterative Deepening Search, IDS) — Tutorial paso a paso

Este notebook explica y ejecuta una implementación de **Búsqueda en Profundidad Iterativa (IDS)** sobre el **mapa de Rumania** (Arad → Bucharest).

IDS combina lo mejor de:
- **DFS (Depth-First Search)**: poco consumo de memoria
- **BFS (Breadth-First Search)**: encuentra soluciones con **menor número de pasos** (si los costos son uniformes)

La idea es ejecutar repetidamente una **Búsqueda con Límite de Profundidad (DLS)** con límites 0, 1, 2, … hasta encontrar el objetivo.



## 0) Importaciones
Para implementar una frontera tipo **pila (LIFO)** de forma eficiente, usamos `collections.deque`.


In [ ]:
from collections import deque

## 1) Clase `Node`

Un **nodo de búsqueda** contiene:
- `state`: el estado actual (ciudad)
- `parent`: referencia al nodo padre (para reconstruir el camino)
- `action`: acción aplicada para llegar a este estado
- `depth`: profundidad del nodo en el árbol de búsqueda (raíz = 0)



In [ ]:
class Node:
    def __init__(self, state, parent=None, action=None, depth=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.depth = depth

    def __repr__(self):
        return f"Node({self.state})"


## 2) Expansión: `expand(problem, node)`

Genera los sucesores del nodo aplicando todas las acciones disponibles desde su estado.


In [ ]:
def expand(problem, node):
    children = []
    for action in problem.actions(node.state):
        child_state = problem.result(node.state, action)
        child_node = Node(
            child_state,
            parent=node,
            action=action,
            depth=node.depth + 1  # incrementa la profundidad para hacer tracking
        )
        children.append(child_node)
    return children


## 3) Evitar ciclos redundantes: `is_cycle(node)`

IDS/DLS (en grafos) puede caer en ciclos si el mapa permite regresar a estados anteriores.

Esta función verifica si el estado del nodo actual ya aparece en su **cadena de ancestros** (es decir, en la misma rama del árbol).


In [ ]:
def is_cycle(node):
    current = node.parent
    while current is not None:
        if current.state == node.state:
            return True
        current = current.parent
    return False


## 4) Búsqueda con Límite de Profundidad (DLS)

DLS es una DFS que **corta** (cutoff) cuando la profundidad excede un límite `limit`.

- Retorna un `Node` si encuentra la meta.
- Retorna `'cutoff'` si se alcanzó el límite en alguna rama sin encontrar meta.
- Retorna `'failure'` si se exploró todo lo permitido sin encontrar meta ni cortes.

**Nota importante:** en tu implementación original se imprimía la frontera en cada iteración. Aquí lo dejamos como opción `debug=True`.



In [ ]:
def depth_limited_search(problem, limit, debug=False):
    frontier = deque([Node(problem.initial)])  # frontera LIFO (pila)
    result = "failure"

    while frontier:
        if debug:
            print("Frontier:", list(frontier))

        node = frontier.pop()

        if problem.is_goal(node.state):
            return node

        if node.depth > limit:
            result = "cutoff"
        else:
            for child in expand(problem, node):
                if not is_cycle(child):
                    frontier.append(child)

    return result


## 5) Búsqueda en Profundidad Iterativa (IDS)

IDS ejecuta DLS para límites crecientes:
- límite = 0, 1, 2, ...

Cuando DLS retorna un nodo (solución) o `'failure'` sin cortes, el algoritmo termina.



In [ ]:
def iterative_deepening_search(problem, max_depth=50, debug=False):
    for depth in range(max_depth):
        result = depth_limited_search(problem, depth, debug=debug)
        if debug:
            print("Resultado con límite", depth, "=>", result)

        # Si no es cutoff, puede ser solución (Node) o failure definitivo
        if result != "cutoff":
            return result

    return None


## 6) Abstracción del problema: clase `Problem`

Agrupa los elementos específicos del dominio:
- `initial`: estado inicial
- `goal`: estado objetivo
- `actions(state)`: acciones disponibles
- `result(state, action)`: transición
- `is_goal(state)`: prueba de objetivo


In [ ]:
class Problem:
    def __init__(self, initial, goal, actions, result, is_goal):
        self.initial = initial
        self.goal = goal
        self.actions = actions
        self.result = result
        self.is_goal = is_goal


## 7) Ejemplo: mapa de Rumania (Arad → Bucharest)

El grafo está representado como un diccionario:
- `actions[ciudad] -> lista de ciudades vecinas`

En este encoding, `result(state, action)` simplemente retorna `action` (la ciudad destino).


In [ ]:
def example_problem():
    initial = "Arad"
    goal = "Bucharest"

    actions = {
        "Arad": ["Zerind", "Timisoara", "Sibiu"],
        "Zerind": ["Oradea", "Arad"],
        "Oradea": ["Zerind", "Sibiu"],
        "Timisoara": ["Arad", "Lugoj"],
        "Lugoj": ["Timisoara", "Mehadia"],
        "Mehadia": ["Lugoj", "Drobeta"],
        "Drobeta": ["Mehadia", "Craiova"],
        "Craiova": ["Drobeta", "Rimnicu Vilcea", "Pitesti"],
        "Sibiu": ["Arad", "Oradea", "Fagaras", "Rimnicu Vilcea"],
        "Fagaras": ["Sibiu", "Bucharest"],
        "Rimnicu Vilcea": ["Sibiu", "Craiova", "Pitesti"],
        "Pitesti": ["Rimnicu Vilcea", "Craiova", "Bucharest"],
        "Bucharest": ["Fagaras", "Pitesti", "Giurgiu", "Urziceni"],
        "Giurgiu": ["Bucharest"],
        "Urziceni": ["Bucharest", "Hirsova", "Vaslui"],
        "Hirsova": ["Urziceni", "Eforie"],
        "Eforie": ["Hirsova"],
        "Vaslui": ["Urziceni", "Iasi"],
        "Iasi": ["Vaslui", "Neamt"],
        "Neamt": ["Iasi"],
    }

    def result(state, action):
        return action

    def is_goal(state):
        return state == goal

    return Problem(initial, goal, lambda s: actions.get(s, []), result, is_goal)


## 8) Reconstrucción del camino solución

Seguimos punteros al padre desde el nodo objetivo hasta la raíz.



In [ ]:
def reconstruir_camino(node):
    path = []
    while node:
        path.append(node.state)
        node = node.parent
    return list(reversed(path))


## 9) Ejecutar IDS

Puedes activar `debug=True` para ver el comportamiento interno (frontera y resultados por límite).


In [ ]:
problem = example_problem()

solution = iterative_deepening_search(problem, max_depth=50, debug=False)

if isinstance(solution, Node):
    path = reconstruir_camino(solution)
    print("Camino solución (IDS):", path)
    print("Número de pasos:", len(path) - 1)
else:
    print("Resultado:", solution)


## 10) Interpretación

- IDS encuentra soluciones con **mínimo número de pasos** (si consideras costos uniformes).
- Consume **mucho menos** memoria que BFS en espacios grandes, porque su frontera en DLS es una pila.
- Repite trabajo (re-expande niveles), pero es un intercambio típico para reducir memoria.

